## Imports

In [1]:
import pandas as pd
import os
from main import list_files_in_folder

## Load data


In [39]:
from dataClass import DataTable

cta_gt_path = 'data/HardTablesR1/DataSets/HardTablesR1/Valid/gt/cta_gt.csv'
cta_gt = pd.read_csv(cta_gt_path, header=None)

tables_path = 'data/HardTablesR1/DataSets/HardTablesR1/Valid/tables'
tables = list_files_in_folder(tables_path)

t = DataTable(tables[15])
print(t.name)
print(cta_gt[cta_gt[0] == '89GM9ZJW'])
print(t.data)

89GM9ZJW
           0  1                                        2
11  89GM9ZJW  0  http://www.wikidata.org/entity/Q1318295
12  89GM9ZJW  1        http://www.wikidata.org/entity/Q5
                                                col0               col1  \
0                                 landesbühne (lenz)     siegfried lenz   
1  narrative of a second expedition to the shores...    john richardson   
2  the life and travels of thomas simpson, the ar...  alexander simpson   

                 col2  
0  hoffmann und campe  
1         john murray  
2     richard bentley  


## CTA on single column


In [24]:
from dotenv import load_dotenv
from dataClass import LamAPI
from langchain_mistralai import ChatMistralAI
load_dotenv()
mistral_api_key = os.getenv("MISTRAL_API_KEY")
model = "open-mixtral-8x7b"
llm = ChatMistralAI(model=model, temperature=0, api_key=mistral_api_key)

In [25]:
t.data.col0.tolist()


['landesbühne (lenz)',
 'narrative of a second expedition to the shores of the polar sea in the years 1825, 1826, and 1827',
 'the life and travels of thomas simpson, the arctic discoverer']

In [5]:
import requests
import json
def LamAPI2(cell_content):
    
    url = 'https://lamapi.hel.sintef.cloud/lookup/entity-retrieval'
    params = {
        'name': f'{cell_content}'.lower(),
        'token': os.getenv("LAMAPI_KEY"),
        'kg': 'wikidata',
        'fuzzy': 'True'
    }
    headers = {'accept': 'application/json'}

    response = requests.get(url, params=params, headers=headers)

    if response.status_code == 200:
        data = response.json()
        # Process the JSON data here
    else:
        print("Error:", response.status_code)
    
    return data

cell_content = t.data.iloc[0,0]
response = LamAPI2(cell_content)

In [26]:
types = []
for cell_content in t.data.iloc[:,0]:
    response = LamAPI2(cell_content)
    for entity in response[cell_content]:
        for ty in entity['types']:
            types.append(ty)
print(len(types))

313


In [29]:
t.generate_ner_labels(llm)
print(t.ner)

{'0': 'NEC', '1': 'NEC', '2': 'NEC'}


In [7]:
import pandas as pd
from collections import Counter

# Create a counter for the ids
counter = Counter(entity['id'] for entity in types)

# Create a unique list of entities based on 'id'
unique_entities = {entity['id']: entity for entity in types}.values()

# Prepare the data for the dataframe
df_data = []
for entity in unique_entities:
    entity_id = entity['id']
    entity_name = entity['name']
    count = counter[entity_id]
    df_data.append({'id': entity_id, 'name': entity_name, 'count': count})

# Create the dataframe
df = pd.DataFrame(df_data)

candidates = {}
for k, (id, name, count) in df.sort_values(by='count', ascending=False).head(25).iterrows():
    candidates[name] = id
list(candidates.keys())

['street',
 'human',
 'architectural structure',
 'Wikimedia disambiguation page',
 'Ortsteil',
 'family name',
 'taxon',
 'locality',
 'Bodendenkmal',
 'natural monument in Germany',
 'human settlement',
 'Wikimedia category',
 'cadastral municipality of Austria',
 'house',
 'non-urban municipality in Germany',
 'hill',
 'mountain',
 None,
 'cultural heritage ensemble',
 'church building',
 'researcher',
 'politician',
 'encyclopedia article',
 'Wikimedia template',
 'municipality seat']

In [8]:
col = 'col0'
cells = list(t.data[col])
prompt = (
    'You have to perform Column Table Association.\n\n'
    f'Column Name: {col}\n'
    f'Column Cells: {cells}\n'
    f'Candidates: {list(candidates.keys())}\n\n'
    'Based on the column name and columns cells choose the correct column type among the candidates and return the chosen candidate in the format [[[choosen_type_name]]].\n'
    'Please provide the response strictly in the format [[[choosen_type_name]]]. Do not include any additional text or explanation.\nExample of your answer:[[[actor]]]'
    'Correct candidate: \n'
    )
print(prompt)

You have to perform Column Table Association.

Column Name: col0
Column Cells: ['predlitz', 'stadtbergen', 'michelstetten', 'ahorn', 'stillfried', 'oberweg']
Candidates: ['street', 'human', 'architectural structure', 'Wikimedia disambiguation page', 'Ortsteil', 'family name', 'taxon', 'locality', 'Bodendenkmal', 'natural monument in Germany', 'human settlement', 'Wikimedia category', 'cadastral municipality of Austria', 'house', 'non-urban municipality in Germany', 'hill', 'mountain', None, 'cultural heritage ensemble', 'church building', 'researcher', 'politician', 'encyclopedia article', 'Wikimedia template', 'municipality seat']

Based on the column name and columns cells choose the correct column type among the candidates and return the chosen candidate in the format [[[choosen_type_name]]].
Please provide the response strictly in the format [[[choosen_type_name]]]. Do not include any additional text or explanation.
Example of your answer:[[[actor]]]Correct candidate: 



In [9]:
out = llm.invoke(prompt)
print(out.content)

Based on the given information, the column name "col0" does not provide any context about the type of the column. However, the column cells contain names of places, which can help us determine the type of the column.

Looking at the candidates, the most appropriate choice for the column type would be "locality" as it refers to a place where a person lives or something is situated.

Therefore, the answer is [[[locality]]].


In [10]:
import re
match = re.search(r'\[\[\[(.*?)\]\]\]', out.content)

if match:
    name = match.group(1)
else:
    name = out.content

print(candidates[name])

Q3257686


In [35]:
for i, col in enumerate(t.data):
    if t.ner[str(i)] == 'NEC':
        for cell_content in list(t.data[col]):
            print(cell_content)

            

landesbühne (lenz)
narrative of a second expedition to the shores of the polar sea in the years 1825, 1826, and 1827
the life and travels of thomas simpson, the arctic discoverer
siegfried lenz
john richardson
alexander simpson
hoffmann und campe
john murray
richard bentley


In [36]:
def C_T_A(t, llm):
    '''
    t = dataClass.DataTable
    llm = langchain model
    '''
    CTA = {}
    for i, col in enumerate(t.data):
        if t.ner == None:
            print("Perform Named Entity Columns Classification first!")
            break
        if t.ner[str(i)] == 'NEC':
            types = []
            for cell_content in list(t.data[col]):
                response = LamAPI2(cell_content)
                for entity in response[cell_content]:
                    for ty in entity['types']:
                        types.append(ty)
            # Create a counter for the ids
            counter = Counter(entity['id'] for entity in types)

            # Create a unique list of entities based on 'id'
            unique_entities = {entity['id']: entity for entity in types}.values()

            # Prepare the data for the dataframe
            df_data = []
            for entity in unique_entities:
                entity_id = entity['id']
                entity_name = entity['name']
                count = counter[entity_id]
                df_data.append({'id': entity_id, 'name': entity_name, 'count': count})

            # Create the dataframe
            df = pd.DataFrame(df_data)

            candidates = {}
            for k, (id, name, count) in df.sort_values(by='count', ascending=False).head(25).iterrows():
                candidates[name] = id
            list(candidates.keys())

            cells = list(t.data[col])
            prompt = (
                'You have to perform Column Table Association.\n\n'
                f'Column Name: {col}\n'
                f'Column Cells: {cells}\n'
                f'Candidates: {list(candidates.keys())}\n\n'
                'Based on the column name and columns cells choose the correct column type among the candidates and return the chosen candidate in the format [[[choosen_type_name]]].\n'
                'Please provide the response strictly in the format [[[choosen_type_name]]]. Do not include any additional text or explanation.\nExample of your answer:[[[actor]]]\n\n'
                'Correct candidate: \n'
                )
            out = llm.invoke(prompt)
            match = re.search(r'\[\[\[(.*?)\]\]\]', out.content)
            if match:
                name = match.group(1)
            else:
                name = out.content
            if name == '' or name == None:
                CTA[str(i)] = 'NIL'
            else:
                CTA[str(i)] = candidates[name]
            
    return CTA


In [38]:
t.cta = C_T_A(t, llm)
print(t.name)
print(t.cta)

89GM9ZJW
{'0': 'Q11812394', '1': 'Q36180', '2': 'Q1320047'}


## CTA


In [ ]:
from tqdm import tqdm
cta_res = {}
for table in tqdm(tables):
    t = DataTable(table)
    print('##################################')
    print(f"Table Name: {t.name}")
    t.generate_ner_labels(llm)
    cta_res[t.name] = C_T_A(t, llm)
    json.dump(cta_res, open('results/HardTablesR1/Valid/CTA/Valid.json', 'w'))
